In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [33]:
df = pd.read_csv('dataset.csv')
df.drop(columns=['Time', 'location', 'File Name', 'Propagation Delay (s)', 'E2E Delay (s)'], inplace=True)
df.head()

,model,processing_delay (ms),core,model_acc,ram
0,yolov5m,2725.300789,half,64.1,1000
1,yolov5m,2869.115829,half,64.1,1000
2,yolov5m,2394.518852,half,64.1,1000
3,yolov5m,2873.452187,half,64.1,1000
4,yolov5m,2371.568680,half,64.1,1000


In [3]:
core_mapping = {
    'half': 0.5,
    'one': 1,
    'two': 2
}

model_mapping = {
    'yolov5n': 'n',
    'yolov5s': 's',
    'yolov5m': 'm'
}

df['core'] = df['core'].replace(core_mapping)
df['model'] = df['model'].replace(model_mapping)
df

,model,processing_delay (ms),core,model_acc,ram
0,m,2725.300789,0.5,64.1,1000
1,m,2869.115829,0.5,64.1,1000
2,m,2394.518852,0.5,64.1,1000
3,m,2873.452187,0.5,64.1,1000
4,m,2371.568680,0.5,64.1,1000
...,...,...,...,...,...
8995,s,553.641796,2.0,56.8,500
8996,s,237.474442,2.0,56.8,500
8997,s,293.733120,2.0,56.8,500
8998,s,485.414267,2.0,56.8,500


In [4]:
percentile_99 = df.groupby(['core', 'model'])['processing_delay (ms)'].quantile(0.99).reset_index()
result = pd.merge(percentile_99, df[['core', 'model', 'model_acc']], on=['core', 'model'], how='left').drop_duplicates()
#result = result[result['processing_delay (ms)'] < 1000]
result

,core,model,processing_delay (ms),model_acc
0,0.5,m,6662.794981,64.1
1000,0.5,n,902.037110,45.7
2000,0.5,s,2015.697604,56.8
3000,1.0,m,2349.166729,64.1
4000,1.0,n,499.338706,45.7
5000,1.0,s,895.282562,56.8
6000,2.0,m,925.280700,64.1
7000,2.0,n,368.760643,45.7
8000,2.0,s,569.763334,56.8


In [18]:
result.sort_values(by=['core', 'model_acc'], ascending=False)

,core,model,processing_delay (ms),model_acc
6000,2.0,m,925.280700,64.1
8000,2.0,s,569.763334,56.8
7000,2.0,n,368.760643,45.7
3000,1.0,m,2349.166729,64.1
5000,1.0,s,895.282562,56.8
4000,1.0,n,499.338706,45.7
0,0.5,m,6662.794981,64.1
2000,0.5,s,2015.697604,56.8
1000,0.5,n,902.037110,45.7


In [6]:
percentile_99_array = percentile_99.to_numpy()
percentile_99_array

array([[0.5, 'm', 6662.794981009999],
       [0.5, 'n', 902.0371103589999],
       [0.5, 's', 2015.69760351],
       [1.0, 'm', 2349.1667293099995],
       [1.0, 'n', 499.33870558199993],
       [1.0, 's', 895.282561806],
       [2.0, 'm', 925.2806996979998],
       [2.0, 'n', 368.76064302],
       [2.0, 's', 569.763333804]], dtype=object)

## Rough Work

In [274]:
df = pd.read_csv('experiment_logs/model_switching_20240618_193221.csv')
df.drop(columns=['time', 'location', 'file_name', 'propagation_delay (s)', 'e2e_delay (s)'], inplace=True)

In [275]:
df

,cid,model_name,processing_delay (ms),core
0,0,yolov5n,737.141132,500
1,1,yolov5s,725.498915,1000
2,2,yolov5m,549.757719,2000
3,0,yolov5n,1009.588718,500
4,1,yolov5s,648.858786,1000
...,...,...,...,...
127,1,yolov5n,453.592777,1000
128,2,yolov5s,224.377155,2000
129,0,yolov5n,890.006304,500
130,1,yolov5n,323.066473,1000


In [261]:
def calculate_sla_stats(dataframe):
    dataframe['sla_violation'] = dataframe['processing_delay (ms)'] > 800
    sla_stats = dataframe.groupby(['cid', 'model_name', 'core']).agg(
                total_records=('processing_delay (ms)', 'size'),
                sla_violations=('sla_violation', 'sum')
            ).reset_index()
    sla_stats['sla_violation_rate'] = sla_stats['sla_violations'] / sla_stats['total_records']
    return sla_stats

        # Calculate SLA statistics for short-term and long-term
short_term_stats = calculate_sla_stats(short_term_df)
long_term_stats = calculate_sla_stats(long_term_df)


/tmp/ipykernel_8326/2756082764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['sla_violation'] = dataframe['processing_delay (ms)'] > 800
/tmp/ipykernel_8326/2756082764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['sla_violation'] = dataframe['processing_delay (ms)'] > 800


In [263]:
long_term_stats

,cid,model_name,core,total_records,sla_violations,sla_violation_rate
0,0,yolov5n,500,44,22,0.5
1,1,yolov5n,1000,33,0,0.0
2,1,yolov5s,1000,11,0,0.0
3,2,yolov5m,2000,11,0,0.0
4,2,yolov5s,2000,33,0,0.0


In [262]:
short_term_stats

,cid,model_name,core,total_records,sla_violations,sla_violation_rate
0,0,yolov5n,500,23,12,0.521739
1,1,yolov5n,1000,23,0,0.000000
2,2,yolov5s,2000,24,0,0.000000


In [271]:
prev_violation =  pd.DataFrame(columns=['cid', 'model_name', 'core', 'total_records', 'sla_violations','sla_violation_rate'])

In [272]:
for service_id in range(3):
    prev_violation = prev_violation.append({
        'cid': service_id,
        'model_name': '',  # Initialize with an empty model name or the initial model name
        'core': 0,  # Initialize with appropriate default values
        'total_records': 0,
        'sla_violations': 0,
        'sla_violation_rate': float('inf')
    }, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [273]:
previous_sla_violations = {service_id: float('inf') for service_id in range(3)}
previous_sla_violations

{0: inf, 1: inf, 2: inf}

In [ ]:
combined_stats = short_term_stats.merge(
            long_term_stats,
            on=['cid', 'model_name', 'core'],
            suffixes=('_short', '_long')
)

In [215]:
model_image = {
        "yolov5n": "babarkhan93/yolotorch-prop:0.3", 
        "yolov5s": "babarkhan93/yolostorch-prop:0.3", 
        "yolov5m": "babarkhan93/yolomtorch:0.3"
    }

In [216]:
model_image

{'yolov5n': 'babarkhan93/yolotorch-prop:0.3',
 'yolov5s': 'babarkhan93/yolostorch-prop:0.3',
 'yolov5m': 'babarkhan93/yolomtorch:0.3'}

In [222]:
for id, (model, image) in enumerate(model_image.items()):
    print(id, model)

0 yolov5n
1 yolov5s
2 yolov5m


In [226]:
images = ["babarkhan93/yolotorch-prop:0.3", "babarkhan93/yolostorch-prop:0.3", "babarkhan93/yolomtorch:0.3"]
versions = ["yolov5n", "yolov5s", "yolov5m"]
